In [5]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
from torchvision import datasets

import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import savemat
from tqdm import tqdm, trange

In [6]:
# the architecture to use
arch = 'resnet50'

# resnet18, alexnet, resnet50

In [7]:
def initialize_model(model_name, use_pretrained=True):
    model = None

    if model_name == "resnet18":
        model = models.resnet18(pretrained=use_pretrained)

    elif model_name == "alexnet":
        model = models.alexnet(pretrained=use_pretrained)

    elif model_name == "resnet50":
        model = models.resnet50(pretrained=use_pretrained)

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model

In [8]:
model = initialize_model(arch)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
## https://kozodoi.me/python/deep%20learning/pytorch/tutorial/2021/05/27/extracting-features.html

##### HELPER FUNCTION FOR FEATURE EXTRACTION

def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

##### REGISTER HOOK
if arch == "alexnet":
    model.features[0].register_forward_hook(get_features('conv1'))
    model.features[3].register_forward_hook(get_features('conv2'))
    model.features[6].register_forward_hook(get_features('conv3'))
    model.features[8].register_forward_hook(get_features('conv4'))
    model.features[10].register_forward_hook(get_features('conv5'))

    model.classifier[1].register_forward_hook(get_features('linear1'))
    model.classifier[4].register_forward_hook(get_features('linear2'))
    model.classifier[6].register_forward_hook(get_features('linear3'))
    
elif arch == "resnet50":
    model.conv1.register_forward_hook(get_features('conv1'))
    model.layer1[2].conv3.register_forward_hook(get_features('conv2'))
    model.layer2[2].conv3.register_forward_hook(get_features('conv3'))
    model.layer3[2].conv3.register_forward_hook(get_features('conv4'))
    model.layer4[2].conv3.register_forward_hook(get_features('conv5'))
    model.fc.register_forward_hook(get_features('linear'))

## ### Test the HYPOTHESIS

In [6]:
main_dir = "C:/Users/ASUS/Desktop/Object vision group - Stefania Bracci/Workplaces/Workplace_11042022/Python_extractFeatures"
data_dir = f"{main_dir}/data"

In [7]:
batch_size = 8

In [8]:
data_transforms = {
    'RSA': trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

print("Initializing Datasets and Dataloaders for RSA...")

# Create training and validation datasets
our_image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x).replace("\\","/"), data_transforms[x]) for x in ['RSA']}
# Create training and validation dataloaders
our_dataloaders_dict = {x: torch.utils.data.DataLoader(our_image_datasets[x], batch_size=batch_size, shuffle=False, num_workers=2) for x in ['RSA']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders for RSA...


In [9]:
## Save the names of images

# for 'RSA' set
nameTmp_dir_objects = f"{main_dir}/data/RSA\\1-objects\\"
len_dir_objects = len(nameTmp_dir_objects)
nameTmp_dir_both = f"{main_dir}/data/RSA\\2-both\\"
len_dir_both = len(nameTmp_dir_both)
nameTmp_dir_scenes = f"{main_dir}/data/RSA\\3-scenes\\"
len_dir_scenes = len(nameTmp_dir_scenes)


RSA_namesImg = []
N = len(our_image_datasets['RSA'].imgs)
for ii in range(0,N):
    nameTmp = our_image_datasets['RSA'].imgs[ii][0]
    if ii<64:
        RSA_namesImg.append(nameTmp[len_dir_objects:-4])
    elif ii<128:
        RSA_namesImg.append(nameTmp[len_dir_both:-4])
    elif ii<192:
        RSA_namesImg.append(nameTmp[len_dir_scenes:-4])

    
# RSA_namesImg 

In [10]:
classNames = our_dataloaders_dict['RSA'].dataset.classes
classNames, len(our_dataloaders_dict['RSA'].dataset)

(['1-objects', '2-both', '3-scenes'], 192)

## # Extract Features

In [11]:
##### FEATURE EXTRACTION LOOP

typeData = 'RSA' # for reference

if arch == "alexnet":
    # placeholders
    conv1 = []
    conv2 = []
    conv3 = []
    conv4 = []
    conv5 = []

    linear1 = []
    linear2 = []
    linear3 = []

    # placeholder for batch features
    features = {}

    for inputs, labels in tqdm(our_dataloaders_dict[typeData]):

        # forward pass
        preds = model(inputs)

        # add feats and preds to lists
        conv1.append(features['conv1'].numpy())
        conv2.append(features['conv2'].numpy())
        conv3.append(features['conv3'].numpy())
        conv4.append(features['conv4'].numpy())
        conv5.append(features['conv5'].numpy())

        linear1.append(features['linear1'].numpy())
        linear2.append(features['linear2'].numpy())
        linear3.append(features['linear3'].numpy())
        
elif arch == "resnet50":
    # placeholders
    conv1 = []
    conv2 = []
    conv3 = []
    conv4 = []
    conv5 = []

    linear = []

    # placeholder for batch features
    features = {}

    for inputs, labels in tqdm(our_dataloaders_dict[typeData]):

        # forward pass
        preds = model(inputs)

        # add feats and preds to lists
        conv1.append(features['conv1'].numpy())
        conv2.append(features['conv2'].numpy())
        conv3.append(features['conv3'].numpy())
        conv4.append(features['conv4'].numpy())
        conv5.append(features['conv5'].numpy())

        linear.append(features['linear'].numpy())

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:41<00:00,  1.73s/it]


In [12]:
##### INSPECT FEATURES

if arch == "alexnet":
    conv1 = np.concatenate(conv1)
    conv2 = np.concatenate(conv2)
    conv3 = np.concatenate(conv3)
    conv4 = np.concatenate(conv4)
    conv5 = np.concatenate(conv5)

    linear1 = np.concatenate(linear1)
    linear2 = np.concatenate(linear2)
    linear3 = np.concatenate(linear3)

    print('- conv1 shape:', conv1.shape)
    print('- conv2 shape:', conv2.shape)
    print('- conv3 shape:', conv3.shape)
    print('- conv4 shape:', conv4.shape)
    print('- conv5 shape:', conv5.shape)

    print('- linear1 shape:', linear1.shape)
    print('- linear2 shape:', linear2.shape)
    print('- linear3 shape:', linear3.shape)
    
elif arch == "resnet50":
    conv1 = np.concatenate(conv1)
    conv2 = np.concatenate(conv2)
    conv3 = np.concatenate(conv3)
    conv4 = np.concatenate(conv4)
    conv5 = np.concatenate(conv5)

    linear = np.concatenate(linear)

    print('- conv1 shape:', conv1.shape)
    print('- conv2 shape:', conv2.shape)
    print('- conv3 shape:', conv3.shape)
    print('- conv4 shape:', conv4.shape)
    print('- conv5 shape:', conv5.shape)

    print('- linear shape:', linear.shape)
    

- conv1 shape: (192, 64, 112, 112)
- conv2 shape: (192, 256, 56, 56)
- conv3 shape: (192, 512, 28, 28)
- conv4 shape: (192, 1024, 14, 14)
- conv5 shape: (192, 2048, 7, 7)
- linear shape: (192, 1000)


In [13]:
resultDic = dict

if arch == "alexnet":
    resultDic = {"dataType": typeData, "RSA_namesImg": RSA_namesImg,
                 "conv1": conv1, "conv2": conv2, "conv3": conv3, "conv4": conv4, "conv5": conv5,
                 "linear1": linear1, "linear2": linear2, "linear3": linear3}
elif arch == "resnet50":
    resultDic = {"dataType": typeData, "RSA_namesImg": RSA_namesImg,
                 "conv1": conv1, "conv2": conv2, "conv3": conv3, "conv4": conv4, "conv5": conv5,
                 "linear": linear}

save_dir = f"{main_dir}/resultDic/"
os.makedirs(save_dir, exist_ok = True)
savemat(f'{save_dir}{typeData}_resultDic_imagenet_{arch}.mat', resultDic)

In [14]:
conv1[55,0,0,:]

array([-2.6403172 , -2.5605586 , -2.270606  , -2.2329824 , -2.212966  ,
       -2.239099  , -2.2469954 , -2.2478147 , -2.2265952 , -2.1808722 ,
       -2.1468241 , -2.1523309 , -2.183265  , -2.181505  , -2.1638916 ,
       -2.1591716 , -2.129959  , -2.103543  , -2.1054652 , -2.1094918 ,
       -2.1050446 , -2.0997262 , -2.076794  , -2.0724282 , -2.0755517 ,
       -2.0596561 , -2.037902  , -2.0504239 , -2.0461552 , -2.0390081 ,
       -2.036217  , -2.0418673 , -2.024305  , -2.0164416 , -2.034613  ,
       -2.0369735 , -2.0104628 , -1.9944062 , -1.9954331 , -1.9954331 ,
       -1.9967449 , -1.9963818 , -1.9949574 , -1.9954038 , -1.9874294 ,
       -1.9602716 , -1.9488008 , -1.9455204 , -1.9460413 , -1.9420956 ,
       -1.9359642 , -1.9296342 , -1.9229785 , -1.9252954 , -1.9350379 ,
       -1.9450889 , -1.9430118 , -1.9347172 , -1.9323473 , -1.9323473 ,
       -1.9326305 , -1.9480798 , -1.9697824 , -1.9671736 , -1.9661672 ,
       -1.9964082 , -1.9945092 , -1.997051  , -2.0059228 , -2.04